In [14]:
# add source to apt
# sudo apt update
# sudo apt install ni-daqmx
import time
import datetime
import pprint
import nidaqmx
import threading
import numpy as np
import nidaqmx.stream_writers

In [32]:
# 1 kHz, in chunks of 100
rate = 1000
samples_per_read = 100
# 10 Hz
rate = 10
samples_per_read = 10
samples = 0
def aquire_as_thread():
    with nidaqmx.Task() as task:
        task.ai_channels.add_ai_voltage_chan("Dev1/ai0")

        task.timing.cfg_samp_clk_timing(rate, sample_mode=nidaqmx.constants.AcquisitionType.CONTINUOUS)

        def callback(task_handle, every_n_samples_event_type,
                     number_of_samples, callback_data):
            #print('Every N Samples callback invoked.')
            data = task.read(number_of_samples_per_channel=samples_per_read)
            with open('nilog.txt','a') as fi:
                fi.write(str(datetime.datetime.now())+'\n'+ ''.join([f'{s:.5f} ' for s in data]) +'\n')
            global samples
            samples +=1
            return 0

        task.register_every_n_samples_acquired_into_buffer_event(
            samples_per_read, callback)

        task.start()            
        while keep_alive:
            print(f' {samples}x{samples_per_read} datapoints', end = '\r')
            time.sleep(1)
            
keep_alive = True 
threading.Thread(target=aquire_as_thread, args=()).start()


In [50]:
# to stop reading thread:
if 0:
    keep_alive = False
    print(f' {samples}x{samples_per_read} datapoints')

 2701x100 datapoints


sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1fdf5d69e40>


In [39]:
import os
def get_last_lines(file, n, sep = ' / '):
    with open(file, 'rb') as f:
        if os.path.getsize(file) > 20000:
            f.seek(-20000, 2)
        data = f.readlines()
        lines = []
        for i in range(n,0,-1):
            lines.append(data[-i].decode("utf-8"))
        lines = sep.join(lines).replace('\r\n', '').strip()
    return lines

In [49]:
lines = get_last_lines('nilog.txt', 2)
print(lines)

2022-09-13 16:48:08.830978 / 0.00009 -0.00087 0.00058 0.00009 0.00025 -0.00007 -0.00007 0.00009 0.00009 -0.00023 0.00041 -0.00023 -0.00007 0.00025 0.00009 0.00009 -0.00023 0.00009 0.00009 0.00025 0.00009 0.00041 0.00025 0.00025 0.00025 -0.00152 0.00074 0.00041 0.00025 0.00041 0.00041 0.00009 0.00025 0.00009 0.00025 -0.00007 0.00009 0.00041 -0.00007 0.00009 0.00041 0.00025 0.00009 0.00025 0.00009 0.00025 -0.00023 0.00009 0.00025 0.00009 0.00009 0.00041 -0.00023 0.00009 0.00009 0.00009 0.00025 0.00041 0.00025 -0.00007 0.00025 0.00009 0.00009 0.00041 0.00025 0.00009 0.00009 0.00009 0.00041 0.00025 -0.00007 0.00025 0.00009 -0.00007 0.00041 0.00025 -0.00023 -0.00007 0.00041 0.00009 0.00009 -0.00023 0.00009 0.00025 0.00009 -0.00023 0.00009 -0.00007 -0.00007 0.00025 0.00025 0.00041 0.00009 0.00009 0.00009 -0.00007 0.00041 0.00009 0.00009 -0.00039


In [47]:
lines = get_last_lines('nilog.txt', 16, '\n')
for l in lines.split('\n')[::2]:
    print(l)

2022-09-13 16:47:48.134028
2022-09-13 16:47:48.233303
2022-09-13 16:47:48.337059
2022-09-13 16:47:48.439916
2022-09-13 16:47:48.532956
2022-09-13 16:47:48.634461
2022-09-13 16:47:48.736565
2022-09-13 16:47:48.827959
